In [12]:
import pandas as pd
import csv as csv
from sklearn.ensemble import RandomForestClassifier

In [13]:
# Load training data
#import os
#os.getcwd()
train_df = pd.read_csv("titanic/train.csv", header=0)


In [14]:
# Convert "Sex" to be a dummy variable (female = 0, Male = 1)
train_df["Gender"] = train_df["Sex"].map({"female": 0, "male": 1}).astype(int)
train_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Gender
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0


In [15]:
# Complement the missing values of "Age" column with average of "Age"
median_age = train_df["Age"].dropna().median()
if len(train_df.Age[train_df.Age.isnull()]) > 0:
  train_df.loc[(train_df.Age.isnull()), "Age"] = median_age


In [16]:
# remove un-used columns
train_df = train_df.drop(["Name", "Ticket", "Sex", "SibSp", "Parch", "Fare", "Cabin", "Embarked", "PassengerId"], axis=1)
train_df.head(3)

,Survived,Pclass,Age,Gender
0,0,3,22.0,1
1,1,1,38.0,0
2,1,3,26.0,0


In [17]:
# Load test data, Convert "Sex" to be a dummy variable
test_df = pd.read_csv("titanic/test.csv", header=0)
test_df["Gender"] = test_df["Sex"].map({"female": 0, "male": 1}).astype(int)

In [18]:
# Complement the missing values of "Age" column with average of "Age"
median_age = test_df["Age"].dropna().median()
if len(test_df.Age[test_df.Age.isnull()]) > 0:
  test_df.loc[(test_df.Age.isnull()), "Age"] = median_age

In [19]:
# Copy test data's "PassengerId" column, and remove un-used columns
ids = test_df["PassengerId"].values
test_df = test_df.drop(["Name", "Ticket", "Sex", "SibSp", "Parch", "Fare", "Cabin", "Embarked", "PassengerId"], axis=1)
test_df.head(3)

,Pclass,Age,Gender
0,3,34.5,1
1,3,47.0,0
2,2,62.0,1


In [34]:
# Predict with "Random Forest"
import numpy as np
train_data = train_df.values
test_data = test_df.values
model = RandomForestClassifier(n_estimators=100)
#output = model.fit(train_data[0::, 1::], train_data[0::, 0]).predict(test_data).astype(int)
output = model.fit(train_data[0::, 1::], train_data[0::, 0]).predict_proba(test_data).astype(np.float32)

In [45]:
print(output[:,0])

[1.         1.         0.43       0.43437698 0.30482414 0.95
 0.5888571  0.99053574 0.6239153  0.93166846 0.43437698 0.99523807
 0.         0.43       0.01       0.12687878 0.9831667  0.93166846
 0.03233333 1.         0.9905     0.5427143  0.04816667 0.98
 0.         1.         0.         0.9184855  0.6565     0.43437698
 0.98       1.         0.23280953 0.03233333 0.9630026  0.89363354
 0.03233333 0.77757937 0.8328636  0.43437698 0.70069045 0.25198483
 1.         0.         0.         0.8328636  0.76469046 0.43437698
 0.01       0.09       0.89       0.99616665 0.         0.
 0.99616665 0.82913095 1.         0.8328636  0.43437698 0.
 1.         0.6471746  0.89363354 0.30482414 0.11       0.35880953
 0.6239153  0.99       0.70607144 0.01       0.22166666 0.93166846
 0.68151706 0.7152355  0.         0.325      0.43437698 0.
 1.         0.22166666 0.295      1.         0.331      0.43437698
 0.99616665 0.43437698 0.03233333 0.6239153  0.03233333 0.03
 0.30482414 0.43437698 0.04816667 0.4

In [49]:
# export result to be "titanic_submit.csv"
submit_file = open("titanic/titanic_submit.csv", "w", newline='')
file_object = csv.writer(submit_file)
file_object.writerow(["PassengerId", "Survived"])
file_object.writerows(zip(ids, output[:,0]))
submit_file.close()